In [ ]:
from lifelines import CoxPHFitter
from pyspark.sql.session import SparkSession, SparkConf
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
import pandas as pd
import seaborn as sns
from lifelines.utils import k_fold_cross_validation

In [ ]:
df = pd.read_parquet('dados_modelo.parquet')

In [ ]:
df = pd.get_dummies(df,drop_first=True, columns=['natureza_juridica', 'porte', 'capital_social_cat'])

In [ ]:
df.columns

In [ ]:
cph = CoxPHFitter()

In [ ]:
df = df.dropna()

In [ ]:
cnpj = df['cnpj']

In [ ]:
df.columns

In [ ]:
df['latitude'] = df['latitude'] + 21.558548
df['longitude'] = df['longitude'] + 45.4394151

In [ ]:
cph.fit(df, duration_col='idade_anos', event_col='fechada', strata=['grande_area'])

In [ ]:
cph.print_summary()

In [ ]:
print(cph.baseline_cumulative_hazard_.columns)

In [ ]:
cph.baseline_cumulative_hazard_[['ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL',
       'AGUA, ESGOTO, ATIVIDADES DE GESTAO DE RESIDUOS E DESCONTAMINACAO',
       'ALOJAMENTO E ALIMENTACAO', 'ARTES, CULTURA, ESPORTE E RECREACAO',
       'ATIVIDADES ADMINISTRATIVAS E SERVICOS COMPLEMENTARES']].plot()

In [ ]:
cph.baseline_cumulative_hazard_[['ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVICOS RELACIONADOS',
       'ATIVIDADES IMOBILIARIAS',
       'ATIVIDADES PROFISSIONAIS, CIENTIFICAS E TECNICAS',
       'COMERCIO, REPARACAO DE VEICULOS AUTOMOTORES E MOTOCICLETAS',
       'CONSTRUCAO', 'EDUCACAO', 'ELETRICIDADE E GAS']].plot()

In [ ]:
cph.print_summary()

In [ ]:
cph.plot()

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
scores = k_fold_cross_validation(cph, df, duration_col='idade_anos', event_col='fechada', k=3, scoring_method="concordance_index")
print(scores)

In [ ]:
ativas = df[df['fechada'] == 0].reset_index()

In [ ]:
hazards = cph.predict_cumulative_hazard(ativas,times=[5,10,25,50],conditional_after=ativas['idade_anos']).T.reset_index()

In [ ]:
hazards.columns

In [ ]:
ativas['risco_5'] = hazards[5.0]
ativas['risco_10'] = hazards[10.0]
ativas['risco_25'] = hazards[25.0]
ativas['risco_50'] = hazards[50.0]


In [ ]:
ativas['tempo_mediano'] = cph.predict_median(ativas, conditional_after=ativas['idade_anos'])

In [ ]:
ativas = ativas[ativas['tempo_mediano'] < 100]

In [ ]:
h3s = pd.read_parquet('./cnpjs_com_h3.parquet/')

In [ ]:
ativas.merge(h3s, on='cnpj').to_csv('./Cox_kepler.csv', index=False, mode='w')